In [ ]:
import os
import time
import requests
import pandas as pd
import numpy as np
import json

api_key = os.environ.get('riot_api_key')
base_url = "https://na1.api.riotgames.com"
solo_duo = "RANKED_SOLO_5x5"

print(api_key)
print(base_url)

RGAPI-2cfc55ea-22b7-4a04-b2e1-14e1ce3afade
https://na1.api.riotgames.com


In [1]:
def getLadder(type) -> pd.DataFrame:
	url = f"{base_url}/lol/league/v4/{type}/by-queue/{solo_duo}?api_key={api_key}"

	response = requests.get(url)

	print(response)

	if response.status_code == 200:
			data = response.json()
			players = data['entries']

			return pd.DataFrame(players)


NameError: name 'pd' is not defined

In [ ]:
def getMatchIDs(puuid):
	url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=40&api_key={api_key}"

	response = requests.get(url)

	if response.status_code == 200:
			data = response.json()

			return data
	else:
		print(response)

In [ ]:
blueTeamId = 100
redTeamId = 200
def getWinner(teams):
	for t in teams:
		if t['teamId'] == blueTeamId:
			return 0 if t['win'] else 1

def getChampionID(participants, side, position):
	for p in participants:
		if p['teamId'] == side and p['teamPosition'] == position:
			return p['championId']
		
def getTeamGoldAt14(participants, match_timeline):
	frames = match_timeline['info']['frames']
	gold = 0

	participant_frames = frames[14]['participantFrames']

	for participant in participants:
		gold += participant_frames[str(participant)]['totalGold']

	return gold


def mapData(match_data, match_timeline):
	blue_participant_id = [1,2,3,4,5]
	red_participant_id = [6,7,8,9,10]

	blue_gold = getTeamGoldAt14(blue_participant_id, match_timeline)
	red_gold = getTeamGoldAt14(red_participant_id, match_timeline)

	blue_atakhan = 0
	red_atakhan = 0

	if 'atakhan' in match_data['info']['teams'][0]['objectives']:
		blue_atakhan = match_data['info']['teams'][0]['objectives']['atakhan']['kills']

	if 'atakhan' in match_data['info']['teams'][1]['objectives']:
		red_atakhan = match_data['info']['teams'][1]['objectives']['atakhan']['kills']

	
	scrubbed_match_data = {
		'matchId': match_data['metadata']['matchId'],
		'gameDurationSeconds': match_data['info']['gameDuration'],
		'championBlueTop': getChampionID(match_data['info']['participants'], blueTeamId, 'TOP'),
		'championBlueJG': getChampionID(match_data['info']['participants'], blueTeamId, 'JUNGLE'),
		'championBlueMid': getChampionID(match_data['info']['participants'], blueTeamId, 'MIDDLE'),
		'championBlueBot': getChampionID(match_data['info']['participants'], blueTeamId, 'BOTTOM'),
		'championBlueSup': getChampionID(match_data['info']['participants'], blueTeamId, 'UTILITY'),
		'championRedTop': getChampionID(match_data['info']['participants'], redTeamId, 'TOP'),
		'championRedJG': getChampionID(match_data['info']['participants'], redTeamId, 'JUNGLE'),
		'championRedMid': getChampionID(match_data['info']['participants'], redTeamId, 'MIDDLE'),
		'championRedBot': getChampionID(match_data['info']['participants'], redTeamId, 'BOTTOM'),
		'championRedSup': getChampionID(match_data['info']['participants'], redTeamId, 'UTILITY'),
		'goldDifference': blue_gold - red_gold, # Negactive number means red side is ahead
		'blueGrubCount': match_data['info']['teams'][0]['objectives']['horde']['kills'],
		'redGrubCount': match_data['info']['teams'][1]['objectives']['horde']['kills'],
		'blueDragonCount': match_data['info']['teams'][0]['objectives']['dragon']['kills'],
		'redDragonCount': match_data['info']['teams'][1]['objectives']['dragon']['kills'],
		'blueBaronCount': match_data['info']['teams'][0]['objectives']['baron']['kills'],
		'redBaronCount': match_data['info']['teams'][1]['objectives']['baron']['kills'],
		'blueAtakhan': blue_atakhan,
		'redAtakhan': red_atakhan,
		'winner': getWinner(match_data['info']['teams']) # 0 = Blue side, 1 = Red Side
	}

	return scrubbed_match_data

In [ ]:
def getAllLadders():
	ladder = pd.concat([getLadder("challengerleagues"), getLadder("grandmasterleagues"), getLadder("masterleagues")])

	return ladder

In [ ]:
def getMatchIDs(ladder):
	match_ids = []
	puuidProcessed = []

	try:
		for index, row in ladder.iterrows():
			matches = getMatchIDs(row.puuid)
			match_ids.extend(matches)
			puuidProcessed.append(row.puuid)

			if (index + 1) % 20 == 0:
				print(len(match_ids))
				time.sleep(30) # Sleep 30 seconds so that it won't hit the rate limit

		print(len(match_ids)) # should be 6000


	except Exception as e: print(e)
	finally:
		df = pd.DataFrame({ 'match_ids': match_ids })
		df2 = pd.DataFrame({ 'puuid': puuidProcessed })

		print(df.shape)
		print(df2.shape)

		df.to_csv("./MatchIDs2.csv")
		df2.to_csv("./ProcessedPuuids2.csv")

In [ ]:
def getMatches(match_ids):
	all_matches = pd.DataFrame()
	bad_match_ids = []
	match_id = ""
	frames = 0
	match_data = None
	match_timeline = None
	try:

		for index, row in match_ids.iterrows():
			match_id = row.match_id

			url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
			response = requests.get(url)
			
			if response.status_code == 200:
				match_data = response.json()
			else:
				print(response)
				continue

			url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"
			response = requests.get(url)

			if response.status_code == 200:
				match_timeline = response.json()
			else:
				print(response)
				continue

			frames = match_timeline['info']['frames']
			if (len(frames) < 15):
				bad_match_ids.append(match_id)
				continue

			match_data_dict = mapData(match_data, match_timeline)

			all_matches = pd.concat([all_matches, pd.DataFrame([match_data_dict])])
			if (index + 1) % 25 == 0:
				print(len(bad_match_ids))
				print(all_matches.shape)				
				time.sleep(90) # Sleep 90 seconds so that it won't hit the rate limit
	
	except Exception as e: 
		print(e)
		print(match_data)
		print(match_timeline)
	finally:
		all_matches.to_csv("./AllMatches2.csv")
		df = pd.DataFrame({ 'match_id': bad_match_ids })
		df.to_csv("./BadMatchIDs2.csv")

In [ ]:
# Remove Duplicates
match_ids = pd.read_csv("MatchIDs.csv")
match_ids = match_ids.drop("Unnamed: 0", axis=1)

print(match_ids.shape)

match_ids = match_ids.drop_duplicates()

print(match_ids.shape)


(236908, 1)
(81542, 1)


In [ ]:
bad_match_ids = pd.read_csv('./BadMatchIDs.csv')

match_ids = match_ids[~match_ids['match_id'].isin(bad_match_ids['match_id'])]

print(match_ids.shape)

(81237, 1)


In [ ]:
all_matches = pd.read_csv('./AllMatches.csv')

match_ids = match_ids[~match_ids['match_id'].isin(all_matches['matchId'])]

print(match_ids.shape)

(64072, 1)


In [ ]:
getMatches(match_ids)

0
(8, 22)
1
(24, 22)
1
(44, 22)
1
(74, 22)
1
(87, 22)
1
(109, 22)
1
(142, 22)
1
(162, 22)
1
(182, 22)
3
(211, 22)
3
(233, 22)
3
(250, 22)
3
(272, 22)
4
(310, 22)
4
(328, 22)
